In [ ]:
import numpy as np
import pandas as pd
import librosa

In [ ]:
PATH_METADATA = "musicnet_metadata.csv"

PATH_DATA = "musicnet/data/"
PATH_LABELS = "musicnet/labels/"

PATH_TRAIN_DATA = "musicnet/train_data/"
PATH_TRAIN_LABELS = "musicnet/train_labels/"
PATH_TEST_DATA = "musicnet/test_data/"
PATH_TEST_LABELS = "musicnet/test_labels/"

In [ ]:
ORIGINAL_SR = 44100
TARGET_SR = 16000
FMIN = librosa.note_to_hz("A0")
FMIN_MIDI_INDEX = librosa.note_to_midi("A0")
N_NOTES = 88
BINS_PER_NOTE = 3
BINS_PER_OCTAVE = 12 * BINS_PER_NOTE
N_BINS = N_NOTES * BINS_PER_NOTE

WINDOW_LENGTH = 2048
HOP_LENGTH = 512

frac_sr = TARGET_SR / ORIGINAL_SR # samples in the csv have to be adjusted because we downsample
sample_indexer = frac_sr / HOP_LENGTH # samples in the csv have to be adjusted because the cqt groups them in windows


# Load wav files, downsample the contents, compute their CQT spectrum, and save it with the same name to the same location as a binary file
def process_wav(id):
    #path_train = PATH_TRAIN_DATA + str(id)
    #path_test = PATH_TEST_DATA + str(id)
    #path = path_train if os.path.isfile(path_train + ".wav") else path_test
    path = PATH_DATA + str(id)

    data_downsampled, sr = librosa.load(path + ".wav", sr=TARGET_SR, res_type='kaiser_best')
    data_cqt = np.abs(librosa.cqt(data_downsampled, sr=sr, hop_length=HOP_LENGTH, fmin=FMIN, n_bins=N_BINS, bins_per_octave=BINS_PER_OCTAVE))
    np.save(path, data_cqt)

    return data_cqt.shape[1]


# Load csv files, transcribe the content to an array of 88 zeros and ones, and save it with the same name to the same location as a binary file
def process_csv(id, n_samples):
    #path_train = PATH_TRAIN_LABELS + str(id)
    #path_test = PATH_TEST_LABELS + str(id)
    #path = path_train if os.path.isfile(path_train + ".csv") else path_test
    path = PATH_LABELS + str(id)

    labels = np.zeros((n_samples, N_NOTES))
    labels_file = np.loadtxt(path + ".csv", delimiter=",", usecols=[0, 1, 3], skiprows=1)

    for line in labels_file:
        initial_frame = round(line[0] * sample_indexer)
        final_frame = round(line[1] * sample_indexer)
        pitch = round(line[2] - FMIN_MIDI_INDEX)
        labels[initial_frame:final_frame, pitch] = 1

    np.save(path, labels)

In [ ]:
# Load metadata from csv file and filter out non-piano music files
metadata = pd.read_csv(PATH_METADATA)
metadata.drop(metadata[metadata["ensemble"] != "Solo Piano"].index, inplace = True)
len(metadata)

In [ ]:
ids = metadata["id"].to_numpy()
print("Files to process: ", len(ids))
for id in ids:
    print(f'Processing {id}...')
    n_samples = process_wav(id)
    process_csv(id, n_samples)

print("Finished")